# Desenvolvendo Modelo de Machine Learning

Agora que temos o _dataset_ com as imagens e as labels podemos começar a desenvovler o modelo. Começamos fazendo pesquisa em relação a outros modelos de ML que façam algo similar e que tecnologias são mais usadas atualmente em visão computacional. Após a pesquisa descobrimos o Detectron2, ele é um framework código aberto desenvolvido pelo Facebook AI Research (FAIR) que fornece uma plataforma de alto desempenho e fácil de usar para a pesquisa de detecção e segmentação de objetos.

<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

O Detectron2 permite a implementação fácil e eficiente de algoritmos de detecção de objetos, incluindo Mask R-CNN. Mask R-CNN é um algoritmo de visão computacional, uma extensão do Faster R-CNN, um algoritmo de detecção de objetos. Enquanto Faster R-CNN é projetado para identificar a localização dos objetos em uma imagem e classificá-los em categorias, Mask R-CNN vai um passo além e adiciona uma terceira tarefa: gerar uma máscara de segmentação para cada instância do objeto detectado. A máscara é basicamente um mapa binário que indica os pixels na imagem que pertencem ao objeto.

Mask R-CNN é bastante útil para tarefas de detecção e _object segmentation_, porque pode fornecer informações mais detalhadas sobre a forma e a localização do objeto. Na tarefa de detecção de telhados e paines solares em imagens de satélite, por exemplo, o Mask R-CNN pode ajudar a identificar a localização exata e a forma de cada telhado, o que pode ser muito útil para a aplicação que queremos, de poder fazer uma estimativa de tamanho.

A segmentação de instâncias (ou _instance segmentation_) é uma tarefa de visão computacional que envolve não apenas a identificação e classificação de objetos em uma imagem (_object detection_), mas também a determinação exata dos pixels que pertencem a cada objeto identificado. Então enquanto a detecção de objetos pode identificar e classificar vários objetos em uma imagem e fornecer uma caixa delimitadora ao redor de cada objeto, a _instance segmentation_ vai além e fornece uma máscara binária para cada objeto, representando exatamente a forma do objeto, o que é importatne para nosso objetivo ja que queremos a forma exata do telhado, e não apenas identifica-los em uma imagem.

Por isso usamos o Mask R-CNN, que é um modelo de rede neural convolucional (CNN) que foi desenvolvido para tarefas de detecção e segmentação de objetos em imagens, _instance segmentation_. Ele é uma extensão do modelo Faster R-CNN, que é usado para detecção de objetos. O Mask R-CNN adiciona uma etapa adicional ao Faster R-CNN para gerar máscaras de segmentação precisas para cada objeto detectado na imagem. Isso permite que o modelo localize objetos com mais precisão e segmente-os com mais precisão do que o Faster R-CNN, fazendo-o mais adequado para nossa aplicação.

O Mask R-CNN é amplamente utilizado em aplicações de visão computacional, e esta inlcuido no _framework_ Detectron2, que por sua vez, é uma plataforma que permite a implementação eficiente de algoritmos de detecção de objetos, incluindo Mask R-CNN. Ele fornece uma infraestrutura robusta e flexível para treinar e executar modelos Mask R-CNN, com suporte para muitas variantes e extensões, e é baseado no Pytorch.

In [ ]:
#Aqui foram apenas algumas configurações e testes ↓

In [ ]:
import sys
print(sys.executable)
print("Hello, World!")

/Users/tiberio/anaconda3/bin/python
Hello, World!


In [ ]:
# Metal Performance Shaders
import torch

if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    mps_device = torch.device("mps")
    print("MPS is available. Device:", mps_device)


MPS is available. Device: mps


In [ ]:
#Deletador2000
from detectron2.data import DatasetCatalog, MetadataCatalog

#remove 'satellite_dataset' de DatasetCatalog e MetadataCatalog
if "satellite_dataset" in DatasetCatalog.list():
    DatasetCatalog.remove("satellite_dataset")
    print ("Deletado com sucesso!")
if "satellite_dataset" in MetadataCatalog.list():
    MetadataCatalog.remove("satellite_dataset")
    print ("Deletado com sucesso!")
else:
    print("Não tinha nada...")

Deletado com sucesso!
Deletado com sucesso!


↓ Primeira versão (funcional) do modelo que batizamos de **SolEngine**. Após instalar o PyTorch e o [Detectron2](https://github.com/facebookresearch/detectron2) nos damos ao Detectron o _path_ para a pasta com nossas imagens para treino e o arquivo JSON no formato COCO. O COCO (Common Objects in Context) é um conjunto de dados de referência muito utilizado em tarefas de visão computacional, como detecção e segmentação de objetos, nesse arquivo contem a lsita de todas as imagens e as _labels_ que fizemos no Label Studio. Depois importamos os modelos do Detectron e escolhemos o que achamos que melhor encaixaria em nossa necessidade, o Mask R-CNN 50 FPN 3x. Depois das importações podemos passar para importante etapa de configurar os hiperparametros. Hiperparâmetros em machine learning são configurações ajustáveis que determinam a estrutura e o desempenho de um algoritmo de aprendizado de máquina. Diferentes dos parâmetros do modelo que o algoritmo aprende sozinho durante o treinamento, os hiperparâmetros devem ser definidos antes do treinamento e geralmente permanecem constantes durante todo o processo. A escolha dos hiperparâmetros pode ter um impacto significativo no desempenho de um modelo de aprendizado de máquina, e encontrar a combinação correta geralmente envolve um processo de experimentação e ajuste, conhecido como otimização de hiperparâmetros. Isso pode ser feito de várias maneiras, incluindo busca em grade (grid search), busca aleatória (random search) e otimização bayesiana, entre outros.

Algum dos hiperparâmetros que usamos foram:

- **Taxa de aprendizado (Learning Rate):** controla o quanto os parâmetros do modelo são ajustados a cada iteração do algoritmo. Uma taxa de aprendizado muito alta pode fazer com que o algoritmo passe pela solução ideal, enquanto uma taxa de aprendizado muito baixa pode tornar o treinamento desnecessariamente lento. É uma das partes mais importantes do treinamento de um modelo de aprendizado de máquina, pois controla a taxa na qual o modelo é atualizado em resposta ao erro que está vendo. Usamos um valor de 0.02 como um ponto de partida considerado razoável, mas pode (e foi) ser ajustado para otimizar a convergência do treinamento.

- **Tamanho do lote (Batch Size):** é o número de exemplos de treinamento usados em uma iteração do algoritmo de otimização. Refere-se ao número de amostras que serão propagadas através da rede ao mesmo tempo. Devido ao baixa resolução das imagens e a grande quantidade de RAM disponivel fomos gradativamente aumentando esse numero com nossos testes.

- **Número de épocas (Number of Epochs)**: define quantas vezes o algoritmo passará pelo conjunto de treinamento inteiro. Mais épocas podem permitir que o modelo aprenda melhor, mas também pode levar a um _overfit_ se o número for muito alto. O número de epochs não é diretamente especificado nas configurações do Detectron2. No entanto, o conceito de epochs é indiretamente controlado pelo parâmetro `cfg.SOLVER.MAX_ITER`. O número de iterações de treinamento precisa ser suficiente para permitir que o modelo convirja, mas não tão alto a ponto de causar _overfit_. O valor ideal pode variar dependendo do tamanho do conjunto de dados de treinamento e da complexidade do problema.

- **RoI Head Batch Size Per Image**: É um parâmetro do modelo no Detectron2 que controla o número de amostras de regiões de interesse (RoIs) que são processadas por imagem em cada mini-lote (batch) durante o treinamento da rede de cabeça (head network) do modelo. As RoIs são regiões propostas pelo modelo que podem conter objetos de interesse, e a rede de cabeça é responsável por classificar e regredir essas RoIs para gerar as previsões finais do modelo. Em áreas urbanas densas, pode haver muitos telhados por imagem, então um valor relativamente alto como 128 pode ser necessário.


Atraves de pesquisa de modelos similares e ajuda do ChatGPT chegamos a os parametros que usamos abaixo para a primeira versão funcional e estavel do nosso modelo, levando em consideração a aplicação do modelo, o dataset e o hardware da maquina.

Começamos de forma mais modesta até encotrar um modelo funcional (abaixo), depois tentamos tecnicas diferentes, como iremos mostrar depois.

O `cfg.MODEL.DEVICE = "cpu"` é uma declaração do detectron2 para que o PyTorch trabalhe apenas com a CPU e ignore a GPU. Isso foi devido ao fato desse modelo ter sido treinado em um Mac, e o Detectron2 foi feito para ter aceleração com GPUs usando CUDA cores, que são especificos de placas da NVIDIA. Esse comando faz com que o programa funcione em sistemas que não contam com GPUs da NVIDIA.

In [ ]:
from detectron2.data.datasets import register_coco_instances

#specify the path to your image and json annotation file (COCO)
register_coco_instances("satellite_dataset", {}, "./labeled_sat_images/result.json", "./labeled_sat_images/images")

from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
#choose a model from model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("satellite_dataset",)
cfg.DATASETS.TEST = ()  #para testes
cfg.DATALOADER.NUM_WORKERS = 2 #número de workers para carregar os dados
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  #initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2 #Batch Size
cfg.SOLVER.BASE_LR = 0.02  #Learning Rate
cfg.SOLVER.MAX_ITER = 2000  #Number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  #my classes: rooftop and solar
cfg.DATALOADER.NUM_WORKERS = 4  #Number of data loading workers (CPU cores, talvez GPUs)

#set device to CPU, n funciona com MPS, so CUDA
cfg.MODEL.DEVICE = "cpu"

from detectron2.engine import DefaultTrainer

#create a Trainer and starts training!
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[05/24 11:49:28 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [05/24 11:49:28 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/24 11:49:28 d2.data.datasets.coco]: Loaded 204 images in COCO format from ./labeled_sat_images/result.json
[05/24 11:49:28 d2.data.build]: Removed 1 images with no usable annotations. 203 images left.
[05/24 11:49:28 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[05/24 11:49:28 d2.data.build]: Using training sampler TrainingSampler
[05/24 11:49:28 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[05/24 11:49:28 d2.data.common]: Serializing 203 elements to byte tensors and concatenating them all ...
[05/24 11:49:28 d2.data.common]: Serialized dataset takes 0.54 MiB
WARNING [05/24 11:49:28 d2.solver.build]: SOLVER.STEPS contains values larger than S

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (2, 256, 1, 1) in

[05/24 11:49:28 d2.engine.train_loop]: Starting training from iteration 0


/Users/tiberio/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.
/Users/tiberio/anaconda3/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/24 11:50:35 d2.utils.events]:  eta: 1:42:48  iter: 19  total_loss: 2.558  loss_cls: 0.8733  loss_box_reg: 0.7599  loss_mask: 0.6861  loss_rpn_cls: 0.1801  loss_rpn_loc: 0.07679    time: 2.9963  last_time: 2.4417  data_time: 0.0645  last_data_time: 0.0012   lr: 0.00039962  
[05/24 11:51:36 d2.utils.events]:  eta: 1:41:46  iter: 39  total_loss: 2.075  loss_cls: 0.5236  loss_box_reg: 0.8161  loss_mask: 0.535  loss_rpn_cls: 0.09698  loss_rpn_loc: 0.08342    time: 3.0297  last_time: 3.2505  data_time: 0.0010  last_data_time: 0.0010   lr: 0.00079922  
[05/24 11:52:31 d2.utils.events]:  eta: 1:37:24  iter: 59  total_loss: 1.625  loss_cls: 0.4084  loss_box_reg: 0.6943  loss_mask: 0.3855  loss_rpn_cls: 0.08105  loss_rpn_loc: 0.07313    time: 2.9435  last_time: 2.2480  data_time: 0.0009  last_data_time: 0.0008   lr: 0.0011988  
[05/24 11:53:29 d2.utils.events]:  eta: 1:35:42  iter: 79  total_loss: 1.485  loss_cls: 0.3688  loss_box_reg: 0.6227  loss_mask: 0.327  loss_rpn_cls: 0.07557  loss_rp

[05/24 12:19:08 d2.utils.events]:  eta: 1:11:51  iter: 599  total_loss: 1.29  loss_cls: 0.3898  loss_box_reg: 0.4546  loss_mask: 0.2693  loss_rpn_cls: 0.0507  loss_rpn_loc: 0.0782    time: 2.9559  last_time: 2.6131  data_time: 0.0010  last_data_time: 0.0008   lr: 0.011988  
[05/24 12:20:05 d2.utils.events]:  eta: 1:10:48  iter: 619  total_loss: 1.32  loss_cls: 0.3535  loss_box_reg: 0.5626  loss_mask: 0.2834  loss_rpn_cls: 0.05055  loss_rpn_loc: 0.07813    time: 2.9512  last_time: 3.2607  data_time: 0.0010  last_data_time: 0.0013   lr: 0.012388  
[05/24 12:21:02 d2.utils.events]:  eta: 1:09:44  iter: 639  total_loss: 1.428  loss_cls: 0.4153  loss_box_reg: 0.5593  loss_mask: 0.3074  loss_rpn_cls: 0.06846  loss_rpn_loc: 0.07912    time: 2.9487  last_time: 2.6163  data_time: 0.0009  last_data_time: 0.0008   lr: 0.012787  
[05/24 12:22:01 d2.utils.events]:  eta: 1:08:41  iter: 659  total_loss: 1.324  loss_cls: 0.3669  loss_box_reg: 0.5647  loss_mask: 0.2699  loss_rpn_cls: 0.06594  loss_rpn_

[05/24 12:47:21 d2.utils.events]:  eta: 0:41:44  iter: 1179  total_loss: 1.275  loss_cls: 0.3127  loss_box_reg: 0.5213  loss_mask: 0.266  loss_rpn_cls: 0.07022  loss_rpn_loc: 0.07105    time: 2.9377  last_time: 3.0518  data_time: 0.0010  last_data_time: 0.0010   lr: 0.02  
[05/24 12:48:20 d2.utils.events]:  eta: 0:40:44  iter: 1199  total_loss: 1.399  loss_cls: 0.3844  loss_box_reg: 0.5638  loss_mask: 0.2893  loss_rpn_cls: 0.06189  loss_rpn_loc: 0.09168    time: 2.9378  last_time: 2.8228  data_time: 0.0010  last_data_time: 0.0009   lr: 0.02  
[05/24 12:49:19 d2.utils.events]:  eta: 0:39:41  iter: 1219  total_loss: 1.314  loss_cls: 0.3348  loss_box_reg: 0.5402  loss_mask: 0.2691  loss_rpn_cls: 0.06441  loss_rpn_loc: 0.08546    time: 2.9381  last_time: 3.1292  data_time: 0.0010  last_data_time: 0.0009   lr: 0.02  
[05/24 12:50:18 d2.utils.events]:  eta: 0:38:42  iter: 1239  total_loss: 1.274  loss_cls: 0.3546  loss_box_reg: 0.5375  loss_mask: 0.2631  loss_rpn_cls: 0.04411  loss_rpn_loc: 

[05/24 13:16:05 d2.utils.events]:  eta: 0:12:22  iter: 1759  total_loss: 1.209  loss_cls: 0.3419  loss_box_reg: 0.4833  loss_mask: 0.2537  loss_rpn_cls: 0.04366  loss_rpn_loc: 0.06368    time: 2.9489  last_time: 2.5940  data_time: 0.0010  last_data_time: 0.0009   lr: 0.02  
[05/24 13:17:03 d2.utils.events]:  eta: 0:11:19  iter: 1779  total_loss: 1.196  loss_cls: 0.3175  loss_box_reg: 0.5175  loss_mask: 0.2802  loss_rpn_cls: 0.04505  loss_rpn_loc: 0.07562    time: 2.9485  last_time: 3.1677  data_time: 0.0009  last_data_time: 0.0009   lr: 0.02  
[05/24 13:18:02 d2.utils.events]:  eta: 0:10:16  iter: 1799  total_loss: 1.258  loss_cls: 0.3377  loss_box_reg: 0.5193  loss_mask: 0.2547  loss_rpn_cls: 0.0422  loss_rpn_loc: 0.06467    time: 2.9481  last_time: 3.1608  data_time: 0.0010  last_data_time: 0.0009   lr: 0.02  
[05/24 13:19:01 d2.utils.events]:  eta: 0:09:14  iter: 1819  total_loss: 1.326  loss_cls: 0.3345  loss_box_reg: 0.5169  loss_mask: 0.2757  loss_rpn_cls: 0.08374  loss_rpn_loc: 

Depois desse primeiro modelo funcional começamos a pesquisar formas de melhora-lo. Uma das melhores formas de melhorar o modelo é fornecendo-lhe mais dados. Devido a o trabalho laborioso que é fazer o processo de _labeling_ dessas imagens e o pouco tempo que o projeto nos da começamos a procurar por formas alternativas de melhorar nosso modelo sem aumentar seu _dataset_.

Uma dessas formas é o ajuste dos hiperparametros, tentamos aumentar o numero de iterações, _batch size_ e _learning rate_. Mas uma das formas mais interessantes que tentamos foi usando otimizadores. Em _machine learning optimizers_ são algoritmos usados para alterar os atributos do modelo de aprendizado de máquina, como pesos e taxas de aprendizado, a fim de reduzir as perdas. Os otimizadores ajudam a obter os resultados mais precisos possíveis. O otimizador que testamos é um bastante popular, o ADAM.

O ADAM (Adaptive Moment Estimation) é um método de otimização que pode ser usado em vez do clássico procedimento de _stochastic gradient descent_ (SGD ou descida de gradiente estocástico) para atualizar os pesos da rede de forma iterativa com base nos dados de treinamento. Adam é um algoritmo popular porque ele supera outros otimizadores em muitos cenários e é menos sensível à inicialização dos parâmetros e a hiperparâmetros, além de ser eficiente computacionalmente.

Ao utilizar o ADAM ja notamos uma grande melhora no nossa curva de perda, que tornou-se menos erratica (esses dados estão disponiveis no notebook de teste).

In [ ]:
from detectron2.data.datasets import register_coco_instances

#specify the path to your image and json annotation file (COCO)
register_coco_instances("satellite_dataset", {}, "./labeled_sat_images/result.json", "./labeled_sat_images/images")

from detectron2.config import get_cfg
from detectron2 import model_zoo

cfg = get_cfg()
#choose a model from model zoo
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("satellite_dataset",)
cfg.DATASETS.TEST = ()  #para testes
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8 #Batch Size
cfg.SOLVER.BASE_LR = 0.001  #Learning Rate  (menor por causa do Adam)
cfg.SOLVER.MAX_ITER = 4000  #Number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  #my classes: rooftop and solar
cfg.DATALOADER.NUM_WORKERS = 4

#set optimizer to Adam
cfg.SOLVER.NAME = "ADAM"

cfg.MODEL.DEVICE = "cpu"

from detectron2.engine import DefaultTrainer

#create a Trainer and starts training!
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[06/11 17:32:57 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [06/11 17:32:57 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[06/11 17:32:57 d2.data.datasets.coco]: Loaded 204 images in COCO format from ./labeled_sat_images/result.json
[06/11 17:32:57 d2.data.build]: Removed 1 images with no usable annotations. 203 images left.
[06/11 17:32:57 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[06/11 17:32:57 d2.data.build]: Using training sampler TrainingSampler
[06/11 17:32:57 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[06/11 17:32:57 d2.data.common]: Serializing 203 elements to byte tensors and concatenating them all ...
[06/11 17:32:57 d2.data.common]: Serialized dataset takes 0.54 MiB
WARNING [06/11 17:32:57 d2.solver.build]: SOLVER.STEPS contains values larger than S

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (2, 256, 1, 1) in

[06/11 17:32:57 d2.engine.train_loop]: Starting training from iteration 0


/Users/tiberio/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.
/Users/tiberio/anaconda3/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


...

[06/12 00:39:40 d2.utils.events]:  eta: 9:12:33  iter: 1179  total_loss: 0.7718  loss_cls: 0.1974  loss_box_reg: 0.2937  loss_mask: 0.2028  loss_rpn_cls: 0.01699  loss_rpn_loc: 0.04934    time: 12.2069  last_time: 11.7650  data_time: 0.0030  last_data_time: 0.0035   lr: 0.001  
[06/12 00:43:25 d2.utils.events]:  eta: 9:07:56  iter: 1199  total_loss: 0.7567  loss_cls: 0.197  loss_box_reg: 0.3001  loss_mask: 0.2036  loss_rpn_cls: 0.01561  loss_rpn_loc: 0.04875    time: 12.1905  last_time: 10.7723  data_time: 0.0032  last_data_time: 0.0027   lr: 0.001  
[06/12 00:47:11 d2.utils.events]:  eta: 9:03:40  iter: 1219  total_loss: 0.7274  loss_cls: 0.1783  loss_box_reg: 0.2883  loss_mask: 0.2013  loss_rpn_cls: 0.01901  loss_rpn_loc: 0.04452    time: 12.1763  last_time: 11.6450  data_time: 0.0029  last_data_time: 0.0031   lr: 0.001  
[06/12 00:50:57 d2.utils.events]:  eta: 8:59:39  iter: 1239  total_loss: 0.7317  loss_cls: 0.1792  loss_box_reg: 0.2882  loss_mask: 0.2009  loss_rpn_cls: 0.01473  l

[06/12 02:29:00 d2.utils.events]:  eta: 7:09:44  iter: 1759  total_loss: 0.5267  loss_cls: 0.1063  loss_box_reg: 0.2172  loss_mask: 0.168  loss_rpn_cls: 0.006705  loss_rpn_loc: 0.03491    time: 11.9107  last_time: 11.3523  data_time: 0.0029  last_data_time: 0.0035   lr: 0.001  
[06/12 02:32:47 d2.utils.events]:  eta: 7:05:33  iter: 1779  total_loss: 0.6227  loss_cls: 0.134  loss_box_reg: 0.249  loss_mask: 0.1886  loss_rpn_cls: 0.008036  loss_rpn_loc: 0.04369    time: 11.9045  last_time: 11.3213  data_time: 0.0031  last_data_time: 0.0032   lr: 0.001  
[06/12 02:36:32 d2.utils.events]:  eta: 7:01:38  iter: 1799  total_loss: 0.5575  loss_cls: 0.119  loss_box_reg: 0.2241  loss_mask: 0.1651  loss_rpn_cls: 0.008774  loss_rpn_loc: 0.037    time: 11.8973  last_time: 11.6986  data_time: 0.0032  last_data_time: 0.0030   lr: 0.001  
[06/12 02:40:18 d2.utils.events]:  eta: 6:57:36  iter: 1819  total_loss: 0.6149  loss_cls: 0.1507  loss_box_reg: 0.2393  loss_mask: 0.1794  loss_rpn_cls: 0.01109  los

[06/12 04:18:21 d2.utils.events]:  eta: 5:16:05  iter: 2339  total_loss: 0.4624  loss_cls: 0.09396  loss_box_reg: 0.1828  loss_mask: 0.1555  loss_rpn_cls: 0.003818  loss_rpn_loc: 0.03079    time: 11.7622  last_time: 11.3430  data_time: 0.0030  last_data_time: 0.0028   lr: 0.001  
[06/12 04:22:06 d2.utils.events]:  eta: 5:12:16  iter: 2359  total_loss: 0.501  loss_cls: 0.1052  loss_box_reg: 0.1872  loss_mask: 0.1583  loss_rpn_cls: 0.005002  loss_rpn_loc: 0.03601    time: 11.7578  last_time: 11.3820  data_time: 0.0030  last_data_time: 0.0027   lr: 0.001  
[06/12 04:25:51 d2.utils.events]:  eta: 5:08:28  iter: 2379  total_loss: 0.4827  loss_cls: 0.09293  loss_box_reg: 0.1865  loss_mask: 0.1568  loss_rpn_cls: 0.004713  loss_rpn_loc: 0.03401    time: 11.7536  last_time: 11.5562  data_time: 0.0030  last_data_time: 0.0032   lr: 0.001  
[06/12 04:29:38 d2.utils.events]:  eta: 5:04:42  iter: 2399  total_loss: 0.5337  loss_cls: 0.1059  loss_box_reg: 0.205  loss_mask: 0.1678  loss_rpn_cls: 0.0056

[06/12 06:07:40 d2.utils.events]:  eta: 3:25:38  iter: 2919  total_loss: 0.4231  loss_cls: 0.07748  loss_box_reg: 0.1698  loss_mask: 0.1443  loss_rpn_cls: 0.00347  loss_rpn_loc: 0.02952    time: 11.6720  last_time: 11.4400  data_time: 0.0031  last_data_time: 0.0028   lr: 0.001  
[06/12 06:11:24 d2.utils.events]:  eta: 3:21:49  iter: 2939  total_loss: 0.4578  loss_cls: 0.08605  loss_box_reg: 0.1852  loss_mask: 0.1487  loss_rpn_cls: 0.003074  loss_rpn_loc: 0.03333    time: 11.6690  last_time: 11.5184  data_time: 0.0030  last_data_time: 0.0039   lr: 0.001  
[06/12 06:15:10 d2.utils.events]:  eta: 3:18:02  iter: 2959  total_loss: 0.4092  loss_cls: 0.07243  loss_box_reg: 0.1603  loss_mask: 0.138  loss_rpn_cls: 0.003095  loss_rpn_loc: 0.02787    time: 11.6665  last_time: 11.4710  data_time: 0.0030  last_data_time: 0.0025   lr: 0.001  
[06/12 06:18:52 d2.utils.events]:  eta: 3:14:13  iter: 2979  total_loss: 0.4062  loss_cls: 0.07852  loss_box_reg: 0.162  loss_mask: 0.139  loss_rpn_cls: 0.0025

[06/12 07:59:30 d2.utils.events]:  eta: 1:35:34  iter: 3499  total_loss: 0.398  loss_cls: 0.07112  loss_box_reg: 0.1504  loss_mask: 0.1375  loss_rpn_cls: 0.002687  loss_rpn_loc: 0.0297    time: 11.6551  last_time: 21.4810  data_time: 0.0044  last_data_time: 0.0065   lr: 0.001  
[06/12 08:04:59 d2.utils.events]:  eta: 1:31:47  iter: 3519  total_loss: 0.3934  loss_cls: 0.06575  loss_box_reg: 0.151  loss_mask: 0.1363  loss_rpn_cls: 0.002342  loss_rpn_loc: 0.02749    time: 11.6821  last_time: 11.9398  data_time: 0.0048  last_data_time: 0.0042   lr: 0.001  
[06/12 08:08:52 d2.utils.events]:  eta: 1:28:03  iter: 3539  total_loss: 0.3806  loss_cls: 0.0703  loss_box_reg: 0.1501  loss_mask: 0.1357  loss_rpn_cls: 0.002731  loss_rpn_loc: 0.0267    time: 11.6820  last_time: 11.8542  data_time: 0.0029  last_data_time: 0.0025   lr: 0.001  
[06/12 08:13:01 d2.utils.events]:  eta: 1:24:14  iter: 3559  total_loss: 0.398  loss_cls: 0.07124  loss_box_reg: 0.1514  loss_mask: 0.1415  loss_rpn_cls: 0.001892

Uma outra tecninca bastatne interessante e que seria muito util para nosso projeto é a _data augmentation_. É uma técnica usada em _machine learning_ para aumentar a quantidade de dados úteis para o treinamento de modelos. Isso é feito através da aplicação de várias transformações que não alteram o significado dos dados originais.

Por exemplo, com as imagens via satélite que estamos usando, algumas técnicas comuns de _data augmentation_ podem incluir rotação, escala, espelhamento, _crop_, alterações de cor, brilho, contraste, etc. Em cada uma dessas alterações, ainda estamos olhando para a mesma imagem original, mas de uma forma ligeiramente diferente.

O objetivo do aumento de dados é tornar o modelo mais robusto para variações nos dados de entrada, permitindo que ele generalize melhor a partir do conjunto de treinamento para dados não vistos. Além disso, ajuda a evitar _overfitting_, que ocorre quando um modelo se ajusta muito bem ao conjunto de treinamento, mas não consegue generalizar bem para novos dados.

Essa é uma tecninca que teria cedo bastatne útil para nós se tivessemos descoberto mais cedo, pois ela resolve o problema que tinhamos de tempo para trabalhar em expandir nosso _dataset_, dessa forma iriamos ter de forma artifical um aumento em nossos dados e por consequencia um modelo mais robusto. Mais infelizmente por falta de tempo não conseguimos chegar a treinar e testar esse modelo. Mas o codigo que estavamos desenvolvendo para isso se encotra abaixo.

In [ ]:
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.data import transforms as T
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer


register_coco_instances("satellite_dataset", {}, "./labeled_sat_images/result.json", "./labeled_sat_images/images")

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("satellite_dataset",)
cfg.DATASETS.TEST = ()  #para testes
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8 #Batch Size
cfg.SOLVER.BASE_LR = 0.001  #Learning Rate  (menor por causa do Adam)
cfg.SOLVER.MAX_ITER = 4000  #Number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  #my classes: rooftop and solar
cfg.DATALOADER.NUM_WORKERS = 4

#set optimizer to Adam
cfg.SOLVER.NAME = "ADAM"

cfg.MODEL.DEVICE = "cpu"

#data augmentation
class Trainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=[
            T.ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'),
            T.RandomFlip(horizontal=True, vertical=False),
            T.RandomRotation(angle=[90, 90], expand=False),
            T.RandomBrightness(0.8, 1.8),
            T.RandomContrast(0.6, 1.3),
            T.RandomSaturation(0.8, 1.4),
            T.RandomLighting(0.7),
        ])
        return build_detection_train_loader(cfg, mapper=mapper)

#use the custom trainer
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


NameError: name 'DefaultPredictor' is not defined